#Instalando dependenências e Bibliotecas

In [ ]:
!pip install gdown

In [ ]:
import os
import math
import gdown
import shutil
import zipfile
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from IPython.display import Image
from sklearn.preprocessing import MinMaxScaler
import itertools

import scipy.stats as stats
from scipy.stats import  norm, pearsonr, spearmanr, ttest_ind, mannwhitneyu, f_oneway, kurtosis, skew

#Variáveis Globais

In [ ]:
# Paleta fornecida
COLORS = [
    '#4FA3A5',  # verde-água
    '#FDAE38',  # laranja
    '#684C41',  # marrom
    '#F75435',  # vermelho
    '#2B6777',  # azul petróleo profundo (PRINCIPAL)
    '#C2A18A',  # bege
    '#FFC7B0',  # rosa claro
    '#3A3F47'   # cinza escuro
]

PRIMARY_BLUE = '#2B6777'

sns.set(style="whitegrid")


In [ ]:
SCORE_COLUMNS = [
    'Total aesthetic score', 'Theme and logic', 'Creativity',
    'Layout and composition', 'Space and perspective',
    'The sense of order', 'Light and shadow', 'Color',
    'Details and texture', 'The overall', 'Mood'
]

SCORE_COLUMNS_NO_TOTAL = [
    col for col in SCORE_COLUMNS if col != 'Total aesthetic score'
]

SCORE_COLUMNS_NO_SoO = [
    col for col in SCORE_COLUMNS_NO_TOTAL if col != 'The sense of order'
]

#Baixar o APDDv2 e suas dependências

Copia o repositório do APDDv2. <br>

Nele tem o arquivo APDDv2-10023.csv: Este arquivo contém anotações para o conjunto de dados APDDv2,
incluindo categorias artísticas, pontuações estéticas totais, pontuações de atributos estéticos e comentários linguísticos para 10.023 imagens.<br>

Nessa pasta também contém o arquivo filesource.csv que tem a origem de cada imagem da base de dados.<br>

A principio não vamos precisar dele

In [ ]:
!git clone https://github.com/BestiVictory/APDDv2.git

O repositório descreve que deve-se fazer o download das imagens ou através do baidu ou através do Google já que o arquivo e grande de mais para ficar no GitHub. A princípio eu fazia o download do arquivo de imagens conforme a próxima célula

In [ ]:
# Defina o diretório temporário onde os arquivos serão armazenados
DATA_FILES = '/content/APDDv2'
OUTPUT_ZIP = f"{DATA_FILES}/APDDv2-images.zip"
UNZIPPED_DIR = f"{DATA_FILES}/APDDv2images"
FILE_IMG_ID = "1ap5dhuEgpPC5PrJozAu2VFmUNIRZrar2"  # ID do arquivo do Google Drive

# Função para fazer o download do arquivo usando gdown
def download_file_from_google_drive(file_id, destination):
    url = f"https://drive.google.com/uc?export=download&id={file_id}"
    gdown.download(url, destination, quiet=False)

# Verifique se o diretório de imagens já foi extraído
if not os.path.isdir(UNZIPPED_DIR):
    print("Baixando o arquivo ZIP...")
    # Crie o diretório se não existir
    os.makedirs(DATA_FILES, exist_ok=True)

    # Baixar o arquivo do Google Drive
    download_file_from_google_drive(FILE_IMG_ID, OUTPUT_ZIP)

    print("Descompactando o arquivo ZIP...")
    os.makedirs(UNZIPPED_DIR, exist_ok=True)
    with zipfile.ZipFile(OUTPUT_ZIP, 'r') as zip_ref:
        zip_ref.extractall(UNZIPPED_DIR)
else:
    print("A pasta já existe. Nenhuma extração necessária.")

Forma de baixar as imagens:
ps: eu não lembro a fonte desse link, mas eu uso isso desde o início do trabalho, então deve ser confiável.

Ele baixa e coloca as imagens no root do colab /content

In [ ]:
!gdown 1eumVJwL8PDajCk9UfMgm7SJjiYyD5C4d

Faz o unzip da pasta colocando as imagens no diretório certo.

In [ ]:
!unzip '/content/APDDv2images.zip' -d '/content/APDDv2'

#Análise exploratória do APDDv2

In [ ]:
csv_path = "/content/APDDv2/APDDv2-10023.csv"

In [ ]:
# Carregar o dataset corretamente como UTF-8
df = pd.read_csv(csv_path, encoding='ISO-8859-1')

# Tratar a coluna 'Artistic Categories' substituindo '*' por ', '
df["Artistic Categories"] = df["Artistic Categories"].str.replace("*", ", ")

# Garantir que a coluna de filename está correta
df["filename"] = df["filename"].astype(str)
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
def expand_artistic_categories(df, column='Artistic Categories', sep=', '):
    """
    Expande a coluna de categorias artísticas em colunas binárias (one-hot encoding).

    Retorna:
    - df_expanded: DataFrame original + colunas de subcategorias
    - category_dummies: DataFrame apenas com as colunas binárias
    - artistic_subcategories: lista com os nomes das subcategorias
    """
    category_dummies = df[column].str.get_dummies(sep=sep)

    # Evita erro ao rodar a célula mais de uma vez no notebook
    df_expanded = (
        df
        .drop(columns=category_dummies.columns, errors='ignore')
        .join(category_dummies)
    )

    artistic_subcategories = category_dummies.columns

    return df_expanded, category_dummies, artistic_subcategories

In [ ]:
df, category_dummies, artistic_subcategories = expand_artistic_categories(df)

In [ ]:
subcategory_proportion = (
    df[artistic_subcategories]
    .sum()
    .sort_values(ascending=False)
    / len(df) * 100
)

display(subcategory_proportion)

plt.figure(figsize=(12, 6))
subcategory_proportion.plot(
    kind='bar',
    color=PRIMARY_BLUE,
    edgecolor='black'
)
plt.title("Proporção de Subcategorias Artísticas")
plt.xlabel("Subcategorias Artísticas")
plt.ylabel("Porcentagem na Base (%)")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


In [ ]:
category_proportion = (
    df['Artistic Categories']
    .value_counts(normalize=True)
    * 100
)

display(category_proportion)

plt.figure(figsize=(10, 5))
category_proportion.plot(
    kind='bar',
    color=PRIMARY_BLUE,
    edgecolor='black'
)
plt.title("Proporção de Artistic Categories")
plt.xlabel("Categorias Artísticas")
plt.ylabel("Porcentagem na Base (%)")
plt.xticks(rotation=45, ha='right')
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(
    data=df[SCORE_COLUMNS],
    color=PRIMARY_BLUE
)
plt.xticks(rotation=45, ha='right')
plt.title("Distribuição das Pontuações (com Total)")
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(
    data=df[SCORE_COLUMNS_NO_TOTAL],
    color=PRIMARY_BLUE
)
plt.xticks(rotation=45, ha='right')
plt.title("Distribuição das Pontuações (sem Total)")
plt.tight_layout()
plt.show()


In [ ]:
missing_by_category = (
    df
    .groupby('Artistic Categories')[SCORE_COLUMNS]
    .apply(lambda x: x.isna().sum())
)

#display(missing_by_category)

In [ ]:
plt.figure(figsize=(14, 6))
sns.heatmap(
    missing_by_category,
    cmap='Reds',
    annot=True,
    fmt='d',
    linewidths=0.5
)
plt.title("Valores Ausentes por Categoria Artística")
plt.xlabel("Colunas de Pontuação")
plt.ylabel("Categorias Artísticas")
plt.tight_layout()
plt.show()


In [ ]:
# Separar subcategorias e aplicar one-hot encoding
df, category_dummies, artistic_subcategories = expand_artistic_categories(df)

# Criar uma tabela com critérios ausentes para cada subcategoria artística
missing_records = []

for category in artistic_subcategories:
    missing_criteria = (
        df.loc[df[category] == 1, SCORE_COLUMNS]
        .isna()
        .sum()
    )

    missing_criteria = missing_criteria[missing_criteria > 0]
    # Contar o número de tokens separados por vírgula na coluna 'Critérios Ausentes'
    if not missing_criteria.empty:
        missing_records.append({
            'Subcategoria Artística': category,
            'Critérios Ausentes': ', '.join(missing_criteria.index),
            'Num Tokens': len(missing_criteria)
        })

missing_df = pd.DataFrame(missing_records)

# Exibir o resultado
print("\nCritérios de avaliação ausentes por subcategoria artística:")
display(missing_df)


In [ ]:
# Histograma para comparar distribuições
sns.histplot(df["Total aesthetic score"], color=PRIMARY_BLUE, label="Original", kde=True, bins=30, alpha=0.5,)

plt.tight_layout()
plt.show()


In [ ]:
# Histograma para comparar distribuições
sns.histplot(df[SCORE_COLUMNS_NO_SoO], color=COLORS, label="Original", kde=True, bins=30, alpha=0.5,)

plt.tight_layout()
plt.show()

In [ ]:
# Criar histogramas para cada coluna
plt.figure(figsize=(15, 12))  # Ajustar o tamanho da figura
for i, col in enumerate(SCORE_COLUMNS_NO_SoO, 1):
    plt.subplot(3, 3, i)  # Criar um grid de subplots 3x3
    sns.histplot(df[col], color=PRIMARY_BLUE, label="Original", kde=True, bins=30, alpha=0.5)
    plt.title(col)  # Título com o nome da coluna

plt.tight_layout()  # Ajustar layout para evitar sobreposição
plt.show()


In [ ]:
# Garante que o azul petróleo seja a primeira cor da paleta
ordered_colors = [PRIMARY_BLUE] + [c for c in COLORS if c != PRIMARY_BLUE]

# Expandir paleta se faltar cor
palette = list(
    itertools.islice(
        itertools.cycle(ordered_colors),
        len(SCORE_COLUMNS_NO_SoO)
    )
)

# Normalização (todas as colunas)
scaler = MinMaxScaler()
df_plot = df[SCORE_COLUMNS_NO_SoO].copy()
df_plot[SCORE_COLUMNS_NO_SoO] = scaler.fit_transform(df_plot[SCORE_COLUMNS_NO_SoO])

plt.figure(figsize=(10, 6))

for col, color in zip(SCORE_COLUMNS_NO_SoO, palette):
    sns.histplot(
        df_plot[col],
        bins=30,
        kde=True,
        alpha=0.5,
        label=col,
        color=color
    )

plt.xlabel("Escala Normalizada (0–1)")
plt.title("Distribuição das Pontuações (Normalizadas)")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
# Matriz de correlação – apenas o triângulo superior
corr = df.select_dtypes(include='number').corr()

mask = np.tril(np.ones_like(corr, dtype=bool))

plt.figure(figsize=(12, 10))
sns.heatmap(
    corr,
    mask=mask,
    cmap=sns.light_palette(PRIMARY_BLUE, as_cmap=True),
    annot=True,
    fmt=".2f",
    linewidths=0.5
)
plt.title("Matriz de Correlação (Triângulo Superior)")
plt.show()
